In [18]:
import os
import random
import shutil
import sys
import yaml
from pathlib import Path
# Change to a different directory
os.chdir(r'C:\Users\SIU856522160\Downloads')
# Get the new default path
os.getcwd()

'C:\\Users\\SIU856522160\\Downloads'

In [19]:
print(100000//3,69863//20,10000//20)

33333 3493 500


In [20]:
num_batches = 20

In [21]:
folder_path = r".\\100k"
source_folder = folder_path

In [22]:
# Destination folder path w     here the selected images will be copied
destination_folder = r".\\data_batches"

In [23]:
def del_folders_and_files(destination_folder):

    if os.path.exists(destination_folder):
        for root, dirs, files in os.walk(destination_folder):
            for dir in dirs:
                subfolder_path = os.path.join(root, dir)
                try:
                    shutil.rmtree(subfolder_path)
                    print(f"Deleted subfolder: {subfolder_path}")
                except OSError as e:
                    print(f"Failed to delete subfolder: {subfolder_path} - {e}")
            for file in files:
                file_path = os.path.join(root, file)
                try:
                    os.remove(file_path)
                    print(f"Deleted file: {file_path}")
                except OSError as e:
                    print(f"Failed to delete file: {file_path} - {e}")
        print("Deleted sub folders and files in the destination folder")
    else:
        print("Destination folder does not exist")



In [24]:
def create_batches(source_folder, num_batches, destination_folder, folder_name):
    if "train" in folder_name.lower() or "val" in folder_name.lower() or "test" in folder_name.lower():
        print(f"Folder is {folder_name}")
        folder_name = folder_name.lower()
    else:
        print("Folder is not train or val")
        return "given the correct folder name"
    
    
    # Skip deletion if folder_name is "val" or "test"
    if folder_name.lower() not in ["val", "test"]:
        del_folders_and_files(destination_folder)

    # del_folders_and_files(destination_folder)

    file_path = os.path.join(rf".//major/dataset/output/output_{folder_name}.txt")

    with open(file_path, "r") as file:
        file_list = file.read().splitlines()
    total_files = len(file_list)

    files_per_batch = total_files // num_batches
    remaining_files = total_files % num_batches

    # Create destination folders for each batch
    for batch_num in range(1, num_batches + 1):
        batch_folder = os.path.join(destination_folder, f"SubData_{batch_num}")
        if not os.path.exists(batch_folder):
            os.makedirs(batch_folder)
        image_destination = os.path.join(batch_folder, "images", folder_name)
        label_destination = os.path.join(batch_folder, "labels", folder_name)
        if not os.path.exists(image_destination):
            os.makedirs(image_destination)
        if not os.path.exists(label_destination):
            os.makedirs(label_destination)

        batch_files = files_per_batch
        if remaining_files > 0:
            batch_files += 1
            remaining_files -= 1

        selected_files = random.sample(file_list, batch_files)

        for file_name in selected_files:
            file_list.remove(file_name)

        file_name_img_save = f"SubData_saved_img_{batch_num}_{folder_name}.txt"
        file_name_txt_save = f"SubData_saved_txt_{batch_num}_{folder_name}.txt"
        copied_images_file_img = open(os.path.join(batch_folder, file_name_img_save), "w")
        copied_images_file_txt = open(os.path.join(batch_folder, file_name_txt_save), "w")

        for file_name in selected_files:
            image_source_path = os.path.join(source_folder, folder_name, file_name)
            image_destination_path = os.path.join(batch_folder, "images", folder_name, file_name)
            print(image_source_path)
            try:
                shutil.copy2(src=image_source_path, dst=image_destination_path)
                print(f"Moved image: {file_name}")
                copied_images_file_img.write(f"{file_name}\n")
            except FileNotFoundError:
                print(f"Image not found: {file_name}")
                continue

            label_name = file_name.split(".")[0] + ".txt"
            label_source_path = os.path.join(source_folder, folder_name, label_name)
            label_destination_path = os.path.join(batch_folder, "labels", folder_name, label_name)
            if os.path.exists(label_source_path):
                try:
                    shutil.copy(src=label_source_path, dst=label_destination_path)
                    print(f"Moved label: {label_name}")
                    copied_images_file_txt.write(f"{label_name}\n")
                except FileNotFoundError:
                    print(f"Label not found: {label_name}")
                    continue
                except OSError as e:
                    print(f"Error occurred while copying label: {label_name} - {e}")
                    continue
                
        # Close the files after writing
        copied_images_file_img.close()
        copied_images_file_txt.close()
        # Create a subfolder named 'sub_date_names' and save the copied_images_file_img in it
        sub_date_names_folder = os.path.join(".\\major\dataset\dataset_file_names", 'sub_date_names',folder_name)
        print(sub_date_names_folder)
        # Skip deletion if folder_name is "val" or "test"
        if not os.path.exists(sub_date_names_folder):
            os.makedirs(sub_date_names_folder)
        if folder_name.lower() not in ["val", "test"] :
            # del_folders_and_files(destination_folder)
            if not any(os.listdir(sub_date_names_folder)): 
                del_folders_and_files(sub_date_names_folder)
        
        shutil.copy2(os.path.join(batch_folder, file_name_img_save), os.path.join(sub_date_names_folder, file_name_img_save))
        # Create a YAML file in the batch folder with the provided configuration
        yaml_data = {
            # 'batch_number': batch_num,
            # 'folder_name': folder_name,
            # 'selected_files': selected_files,
            'names': [
                "person", "rider", "car", "truck", "bus", "train", "motor", "bike", "traffic light", "traffic sign"
            ],
            'path': './data/vech',
            'train': f'./SubData_saved_img_{batch_num}_train.txt',
            'val': f'./SubData_saved_img_{batch_num}_val.txt',
            'nc': 10,
            
            
        }
        # Reverse the order of keys before writing to the YAML file
        yaml_data = {k: yaml_data[k] for k in reversed(yaml_data)}
        yaml_file_path = os.path.join(batch_folder, f"batch_info_{batch_num}.yaml")
        with open(yaml_file_path, 'w') as yaml_file:
            yaml.dump(yaml_data, yaml_file,default_flow_style=False)
            # yaml.dump(yaml_data, yaml_file)

In [25]:
# # create_batches(source_folder, num_batches, destination_folder,folder_name="Train")
# with open(r'.\\major\logs\batches_create_train.txt', 'w') as file:
#     # Redirect the standard output to the file
#     sys.stdout = file
    
#     create_batches(source_folder, num_batches, destination_folder,folder_name="Train")

#     # Reset the standard output to the console
#     sys.stdout = sys.__stdout__

In [26]:
# with open(r'.\\major\logs\batches_create_val.txt', 'w') as file:
#     # Redirect the standard output to the file
#     sys.stdout = file
    
#     create_batches(source_folder, num_batches, destination_folder,folder_name="val")

#     # Reset the standard output to the console
#     sys.stdout = sys.__stdout__

In [27]:
# with open(r'.\\major\logs\batches_create_test.txt', 'w') as file:
#     # Redirect the standard output to the file
#     sys.stdout = file
    
#     create_batches(source_folder, num_batches, destination_folder,folder_name="test")

#     # Reset the standard output to the console
#     sys.stdout = sys.__stdout__

In [28]:
def count_folders(directory):
    folder_count = sum(1 for item in Path(directory).iterdir() if item.is_dir())
    return folder_count

import shutil
import random
from pathlib import Path

def get_subfolder_names(directory):
    return [entry.name for entry in directory.iterdir() if entry.is_dir()]

def dev_move(source, destination_folder, number_of_devices):
    directory_path = Path.cwd() / source 
    directories = get_subfolder_names(directory_path)
    num_directories = len(directories)
    number_of_groups = number_of_devices  # Adjust this to your desired number of groups

    directories_per_folder = num_directories // number_of_devices
    remaining_directories = num_directories % number_of_devices
    device_folders = [f"Device_{i+1}" for i in range(number_of_devices)]
    
    grouped_folders = [[] for _ in range(number_of_groups)]

    # Distribute devices evenly among groups
    for i, folder in enumerate(device_folders):
        group_index = i % number_of_groups
        grouped_folders[group_index].append(folder)

    for i, directory in enumerate(directories):
        source_path = directory_path / directory

        if i >= num_directories - remaining_directories:
            destination_folder_name = random.choice(device_folders)
        else:
            device_number = device_folders[i // directories_per_folder]
            destination_folder_name = f"{device_number}"

        destination_folder_path = Path(destination_folder) / destination_folder_name
        destination_folder_path.mkdir(parents=True, exist_ok=True)

        destination_path = destination_folder_path / directory
        print(f"Copying {directory} to {destination_path}")
        shutil.copytree(str(source_path), str(destination_path))

    print("Batch folders moved to separate destination folders successfully.")



In [30]:
# with open(r'.\major\logs\batches_create_val.txt', 'w') as file:
#     # Redirect the standard output to the file
#     sys.stdout = file
    
#     create_batches_val(source_folder, num_batches, destination_folder,folder_name="train")

#     # Reset the standard output to the console
#     sys.stdout = sys.__stdout__

In [31]:
# def create_batches_val(source_folder, num_batches, destination_folder, folder_name):
#     if "train" in folder_name.lower() or "val" in folder_name.lower():
#         print(f"Folder is {folder_name}")
#         folder_name = folder_name.lower()
#     else:
#         print("Folder is not train or val")
#         return "Given the correct folder name"

#     # del_folders_and_files(destination_folder)

#     file_path = os.path.join(rf".//major/dataset/output/output_{folder_name}.txt")

#     with open(file_path, "r") as file:
#         file_list = file.read().splitlines()
#     total_files = len(file_list)
#     files_per_batch = total_files // num_batches
#     remaining_files = total_files % num_batches

#     for batch_num in range(1, num_batches + 1):
#         batch_folder = os.path.join(destination_folder, f"SubData_{batch_num}")
#         if not os.path.exists(batch_folder):
#             os.makedirs(batch_folder)
#         image_destination = os.path.join(batch_folder, "images", folder_name)
#         label_destination = os.path.join(batch_folder, "labels", folder_name)
#         if not os.path.exists(image_destination):
#             os.makedirs(image_destination)
#         if not os.path.exists(label_destination):
#             os.makedirs(label_destination)

#         batch_files = files_per_batch
#         if remaining_files > 0:
#             batch_files += 1
#             remaining_files -= 1

#         selected_files = random.sample(file_list, batch_files)

#         file_name_img_save = f"SubData_saved_img_{batch_num}_{folder_name}.txt"
#         file_name_txt_save = f"SubData_saved_txt_{batch_num}_{folder_name}.txt"
#         copied_images_file_img = open(os.path.join(batch_folder, file_name_img_save), "w")
#         copied_images_file_txt = open(os.path.join(batch_folder, file_name_txt_save), "w")

#         for file_name in selected_files:
#             image_source_path = os.path.join(source_folder, folder_name, file_name)
#             image_destination_path = os.path.join(batch_folder, "images", folder_name, file_name)

#             try:
#                 shutil.copy2(src=image_source_path, dst=image_destination_path)
#                 print(f"Moved image: {file_name}")
#                 copied_images_file_img.write(f"{file_name}\n")
#             except FileNotFoundError:
#                 print(f"Image not found: {file_name}")
#                 continue

#             label_name = file_name.split(".")[0] + ".txt"
#             label_source_path = os.path.join(source_folder, folder_name, label_name)
#             label_destination_path = os.path.join(batch_folder, "labels", folder_name, label_name)

#             if os.path.exists(label_source_path):
#                 try:
#                     shutil.copy(src=label_source_path, dst=label_destination_path)
#                     print(f"Moved label: {label_name}")
#                     copied_images_file_txt.write(f"{label_name}\n")
#                 except FileNotFoundError:
#                     print(f"Label not found: {label_name}")
#                     continue
#                 except OSError as e:
#                     print(f"Error occurred while copying label: {label_name} - {e}")
#                     continue

#         copied_images_file_img.close()
#         copied_images_file_txt.close()
# create_batches_val(source_folder, num_batches, destination_folder,folder_name="val")

In [32]:
# def create_batches_test(source_folder, num_batches, destination_folder, folder_name):
#     print(folder_name)
#     if "train" in folder_name.lower() or "val" in folder_name.lower() or "test" in folder_name.lower():
#         print(f"Folder is {folder_name}")
#         folder_name = folder_name.lower()
#     else:
#         print("Folder is not train or val")
#         return "Given the correct folder name"

#     # del_folders_and_files(destination_folder)

#     file_path = os.path.join(rf".//major/dataset/output/output_{folder_name}.txt")

#     with open(file_path, "r") as file:
#         file_list = file.read().splitlines()
#     total_files = len(file_list)
#     files_per_batch = total_files // num_batches
#     remaining_files = total_files % num_batches

#     for batch_num in range(1, num_batches + 1):
#         batch_folder = os.path.join(destination_folder, f"SubData_{batch_num}")
#         if not os.path.exists(batch_folder):
#             os.makedirs(batch_folder)
#         image_destination = os.path.join(batch_folder, "images", folder_name)
#         # label_destination = os.path.join(batch_folder, "labels", folder_name)
#         if not os.path.exists(image_destination):
#             os.makedirs(image_destination)
#         # if not os.path.exists(label_destination):
#         #     os.makedirs(label_destination)

#         batch_files = files_per_batch
#         if remaining_files > 0:
#             batch_files += 1
#             remaining_files -= 1

#         selected_files = random.sample(file_list, batch_files)

#         file_name_img_save = f"SubData_saved_img_{batch_num}_{folder_name}.txt"
#         # file_name_txt_save = f"SubData_saved_txt_{batch_num}_{folder_name}.txt"
#         copied_images_file_img = open(os.path.join(batch_folder, file_name_img_save), "w")
#         # copied_images_file_txt = open(os.path.join(batch_folder, file_name_txt_save), "w")

#         for file_name in selected_files:
#             image_source_path = os.path.join(source_folder, folder_name, file_name)
#             image_destination_path = os.path.join(batch_folder, "images", folder_name, file_name)

#             try:
#                 shutil.copy2(src=image_source_path, dst=image_destination_path)
#                 print(f"Moved image: {file_name}")
#                 copied_images_file_img.write(f"{file_name}\n")
#             except FileNotFoundError:
#                 print(f"Image not found: {file_name}")
#                 continue

#             # label_name = file_name.split(".")[0] + ".txt"
#             # label_source_path = os.path.join(source_folder, folder_name, label_name)
#             # label_destination_path = os.path.join(batch_folder, "labels", folder_name, label_name)

#             # if os.path.exists(label_source_path):
#             #     try:
#             #         shutil.copy(src=label_source_path, dst=label_destination_path)
#             #         print(f"Moved label: {label_name}")
#             #         copied_images_file_txt.write(f"{label_name}\n")
#             #     except FileNotFoundError:
#             #         print(f"Label not found: {label_name}")
#             #         continue
#             #     except OSError as e:
#             #         print(f"Error occurred while copying label: {label_name} - {e}")
#             #         continue

#         copied_images_file_img.close()
#         # copied_images_file_txt.close()
# create_batches_test(source_folder, num_batches, destination_folder,folder_name="test")

In [36]:
destination_folder_vech =  r".//vech" # Replace with the path to the destination folder
del_folders_and_files(destination_folder_vech)

Deleted subfolder: .//vech\Device_1
Deleted subfolder: .//vech\Device_2
Deleted subfolder: .//vech\Device_3
Deleted sub folders and files in the destination folder


In [37]:
dev_move(source="data_batches", destination_folder="vech",  number_of_devices=3)

Copying SubData_1 to vech\Device_1\SubData_1
Copying SubData_10 to vech\Device_1\SubData_10
Copying SubData_11 to vech\Device_1\SubData_11
Copying SubData_12 to vech\Device_1\SubData_12
Copying SubData_13 to vech\Device_1\SubData_13
Copying SubData_14 to vech\Device_1\SubData_14
Copying SubData_15 to vech\Device_2\SubData_15
Copying SubData_16 to vech\Device_2\SubData_16
Copying SubData_17 to vech\Device_2\SubData_17
Copying SubData_18 to vech\Device_2\SubData_18
Copying SubData_19 to vech\Device_2\SubData_19
Copying SubData_2 to vech\Device_2\SubData_2
Copying SubData_20 to vech\Device_3\SubData_20
Copying SubData_3 to vech\Device_3\SubData_3
Copying SubData_4 to vech\Device_3\SubData_4
Copying SubData_5 to vech\Device_3\SubData_5
Copying SubData_6 to vech\Device_3\SubData_6
Copying SubData_7 to vech\Device_3\SubData_7
Copying SubData_8 to vech\Device_1\SubData_8
Copying SubData_9 to vech\Device_2\SubData_9
Batch folders moved to separate destination folders successfully.


In [35]:
number_of_devices = 20  # Adjust this to your desired number of devices
number_of_groups = 3  # Adjust this to your desired number of groups

device_folders = [f"Device_{i+1}" for i in range(number_of_devices)]
grouped_folders = [[] for _ in range(number_of_groups)]

# Distribute devices evenly among groups
for i, folder in enumerate(device_folders):
    group_index = i % number_of_groups
    grouped_folders[group_index].append(folder)

# Now, grouped_folders will contain the devices distributed among groups
for i, group in enumerate(grouped_folders):
    print(f"Group {i+1}: {group}")


Group 1: ['Device_1', 'Device_4', 'Device_7', 'Device_10', 'Device_13', 'Device_16', 'Device_19']
Group 2: ['Device_2', 'Device_5', 'Device_8', 'Device_11', 'Device_14', 'Device_17', 'Device_20']
Group 3: ['Device_3', 'Device_6', 'Device_9', 'Device_12', 'Device_15', 'Device_18']


In [ ]:
Copying SubData_1 to vech\Device_1\SubData_1
Copying SubData_10 to vech\Device_1\SubData_10
Copying SubData_11 to vech\Device_1\SubData_11
Copying SubData_12 to vech\Device_1\SubData_12
Copying SubData_13 to vech\Device_1\SubData_13
Copying SubData_14 to vech\Device_1\SubData_14
Copying SubData_15 to vech\Device_2\SubData_15
Copying SubData_16 to vech\Device_2\SubData_16
Copying SubData_17 to vech\Device_2\SubData_17
Copying SubData_18 to vech\Device_2\SubData_18
Copying SubData_19 to vech\Device_2\SubData_19
Copying SubData_2 to vech\Device_2\SubData_2
Copying SubData_20 to vech\Device_3\SubData_20
Copying SubData_3 to vech\Device_3\SubData_3
Copying SubData_4 to vech\Device_3\SubData_4
Copying SubData_5 to vech\Device_3\SubData_5
Copying SubData_6 to vech\Device_3\SubData_6
Copying SubData_7 to vech\Device_3\SubData_7
Copying SubData_8 to vech\Device_1\SubData_8
Copying SubData_9 to vech\Device_2\SubData_9
Batch folders moved to separate destination folders successfully.